# 0 - Setup do Sistema

## 0.1 - Importa os módulos utilizados

In [ ]:
import collections
import kaleido
import matplotlib
from unidecode import unidecode
import pandas as pd
import string
import decimal
import requests
import numpy as np
from tabulate import tabulate
from fuzzywuzzy import process
import gspread
import tweepy 
from tweepy import OAuthHandler
import time
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import sys
import argparse
import json
import logging
import codecs
import os.path
from os.path import join as pjoin
import glob
import json
from io import StringIO
from io import open
from urllib.request import urlopen
from itertools import chain
import os
import csv
import regex as re
from collections import Counter
# import re as regex
# import regex as re
# import re
from tabulate import tabulate
from csv import DictReader
import itertools
import networkx as nx
from networkx.algorithms.community import girvan_newman
from networkx.algorithms.community import greedy_modularity_communities
from networkx.readwrite import json_graph
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from collections import Counter
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as chs
import cufflinks as cf
import plotly.express as px
from wordcloud import WordCloud, ImageColorGenerator

import time
from requests.exceptions import HTTPError
from urllib.request import urlopen
import urllib
import json

import ijson
from datetime import datetime
import glob
# import matplotlib.pyplot as plt, mpld3
from dateutil.parser import parse


%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

## 0.2 - Carrega a base de dados

In [ ]:
listaDeputados = pd.read_csv("db-listaDeputados.csv")
print(listaDeputados.columns)
# pd.set_option('max_columns', None)
# listaDeputados.head(3)


## 0.3 - Formata rótulos das base de dados para as tabelas

In [ ]:
keepCol = ['GENERO', 'COR_RACA', 'ESTADO_CIVIL','ESCOLARIDADE', 'OCUPACAO_ANTERIOR','FAIXA_ETARIA','PARTIDO_ATUAL', 'IDEOLOGIA_PARTIDO_ATUAL','MILIONARIO','TEM_TWITTER','ESTADO_ELEICAO','REGIAO_ELEICAO','ST_REELEICAO', 'AGRUPAMENTO_IDADE','AGRUPAMENTO_BENS','AGRUPAMENTO_VOTOS','AGRUPAMENTO_SEGUIDORES', 'AGRUPAMENTO_BALANCO_SEGUIDORES','AGRUPAMENTO_NUMERO_TWEETS', 'AGRUPAMENTO_BALANCO_TWEETS']

listaDeputados_F = listaDeputados[keepCol]

listaDeputados_F = listaDeputados_F.rename(columns={'GENERO':'Gênero','COR_RACA':'Cor/Raça','ESTADO_CIVIL':'Estado Civil','ESCOLARIDADE':'Escolaridade','OCUPACAO_ANTERIOR':'Ocupação anterior','FAIXA_ETARIA':'Faixa etária','PARTIDO_ATUAL':'Partido','IDEOLOGIA_PARTIDO_ATUAL':'Ideologia','MILIONARIO':'Maioria milionária','TEM_TWITTER':'Tem twitter','ESTADO_ELEICAO':'Estado','REGIAO_ELEICAO':'Região','ST_REELEICAO':'Reeleito','AGRUPAMENTO_IDADE':'AG Idade', 'AGRUPAMENTO_VOTOS':'AG Votos','AGRUPAMENTO_SEGUIDORES': 'AG Seguidores', 'AGRUPAMENTO_BALANCO_SEGUIDORES':'AG Balanço de Seguidores','AGRUPAMENTO_NUMERO_TWEETS':'AG Número de Tweets','AGRUPAMENTO_BALANCO_TWEETS':'AG Balanço de Tweets','AGRUPAMENTO_BENS':'AG Bens'})

# 1 - Análise exploratória

## 1.1 - Levanta coluna generica para as funções de agrupamento

In [ ]:
listaDeputados_F['new'] = 'a'

## 1.2 - Filtra a base de dados por deputados gaúchos

In [ ]:
listaDeputados_rs = listaDeputados_F[(listaDeputados_F['Estado']=='RS')]

## 1.3 - Cria tabela de características gerais

In [ ]:
numero = listaDeputados.value_counts()
percentual = round(listaDeputados.value_counts(normalize=True)*100,2)
matriz = {'Número':numero,'Percentual':percentual}
stats_geral = pd.DataFrame(data=matriz)
moda_geral = listaDeputados_F.groupby('new').agg(lambda x: x.mode())
info_geral = listaDeputados_F.groupby('new').agg(lambda x: x.value_counts().index[0] + f"\n({x.value_counts().iloc[0]} || {round(x.value_counts().iloc[0]/len(x)*100,2)}%)")
# moda_geral.transpose()
# info_geral.transpose().to_clipboard()
info_geral.transpose()

## 1.4 - Cria a tabela de características dos deputados gaúchos

In [ ]:
numero = listaDeputados_rs.value_counts()
percentual = round(listaDeputados_rs.value_counts(normalize=True)*100,2)
matriz = {'Número':numero,'Percentual':percentual}
stats_rs = pd.DataFrame(data=matriz)
moda_rs = listaDeputados_rs.groupby('new').agg(lambda x: x.mode())

info_rs = listaDeputados_rs.groupby('new').agg(lambda x: x.value_counts().index[0] + f"\n({x.value_counts().iloc[0]} || {round(x.value_counts().iloc[0]/len(x)*100,2)}%)")

info_rs.transpose().to_clipboard()
# info_rs.transpose()

## 1.5 - Gera gráfico de deputados por estado

In [ ]:
deputados_estados = listaDeputados['ESTADO_ELEICAO'].value_counts().sort_values( ascending=True)



# deputados_estados = pd.DataFrame(list(zip(titulos,duracao)), index=titulos).sort_values(0, ascending=False)
                            
                            
top = deputados_estados.plot(kind='barh', 
                        colormap='tab10', 
                        figsize=(11, 8),width=0.8)

for i, v in enumerate(list(deputados_estados)):
    top.text(v+0.4, i-0.25, v ,fontsize=12, color='black')

plt.tick_params(axis='y', which='major', labelsize=14)
    
# top.margins(x=1)
# top.spines['right'].set_visible(False)
# top.spines['top'].set_visible(False)
# top.spines['bottom'].set_visible(False)
    
# top.get_legend().remove()            

# plt.rcParams.update(IPython_default);
# plt.style.use("fivethirtyeight")
plt.grid(color='b', linestyle='-', linewidth=.1)    
plt.savefig("novos-graficos/Figura x - Quantidade de Deputados Federais por Estado.jpg",bbox_inches = 'tight')

# 2 - Consistência Temática

## 2.1 - Carrega bases de dados

In [ ]:
# pd.set_option('max_columns', None)
listaDeputados = pd.read_csv("db-listaDeputados.csv")
proposicoes = pd.read_csv("db-proposicoes.csv")
tweetsDeputados = pd.read_csv("tweetsDeputados.csv", sep=";")



# tweetsDeputados_100 = pd.read_csv("tweetsDeputados-T100.csv", sep=";")
# grafo_mt = pd.read_csv("grafo-mt.csv")
# grafo_rt = pd.read_csv("grafo-rt.csv")
# grafo_hashtags = pd.read_csv("grafo-hashtags.csv")
# grafo_semantico = pd.read_csv("grafo-semantico-T100.csv")

# print(listaDeputados.columns)
# listaDeputados.head(3)
# list(proposicoes.columns)
# # proposicoes.head(3)

## 2.2 - Cria base de dados da autoria das proposições

In [ ]:
%%capture
PLs = pd.DataFrame(proposicoes[proposicoes['DESC_PROPOSICAO']=='Projeto de Lei'][proposicoes["TIPO_AUTOR"]=="Deputado"])
# proposicoes.shape[0]
# PLs.shape[0]
# list(PLs.columns)
# PLs.head(3)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = PLs['NOME_AUTOR'].astype('str').str.split(',').map(len)
# PLs.shape[0]
# lens
autores = pd.DataFrame({
    'NOME_AUTOR': chainer(PLs['NOME_AUTOR'].astype("str")), 
    'ID_AUTOR': np.repeat(PLs['ID_AUTOR'], lens),
    'TITULO_PROPOSICAO': np.repeat(PLs['TITULO_PROPOSICAO'], lens),
    'TEMA': np.repeat(PLs['TEMA'], lens),
    'PALAVRAS_CHAVE': np.repeat(PLs['PALAVRAS_CHAVE'], lens),
    'EMENTA': np.repeat(PLs['EMENTA'], lens),
    'DATA_APRESENTACAO': np.repeat(PLs['DATA_APRESENTACAO'], lens),
    'DESC_TRAMITACAO': np.repeat(PLs['DESC_TRAMITACAO'], lens),
    'DESC_SITUACAO': np.repeat(PLs['DESC_SITUACAO'], lens),
    'ID_PROPOSICAO': np.repeat(PLs['ID_PROPOSICAO'], lens),
    'PROPOSICAO_BASE': np.repeat(PLs['PROPOSICAO_BASE'], lens),
    'URL_PROPOSICOES': np.repeat(PLs['URL_PROPOSICOES'], lens),
    'TEOR_URL': np.repeat(PLs['TEOR_URL'], lens)
})
# autores.shape[0]
# autores.reset_index(level=0, inplace=True)
# autores.head(3)



### 2.3 - Conta e agrupa proposições

In [ ]:
autores['N_PLS'] = 1

autores_G = autores.astype("str").groupby(["NOME_AUTOR"],as_index=False).agg({
    'NOME_AUTOR': "first",
    'N_PLS': 'count',
    'TITULO_PROPOSICAO':lambda tags: ','.join(tags),
    'TEMA':lambda tags: ','.join(tags),
    'PALAVRAS_CHAVE':lambda tags: ','.join(tags),
    'EMENTA':lambda tags: ','.join(tags),
    'DATA_APRESENTACAO':lambda tags: ','.join(tags),
    'DESC_TRAMITACAO':lambda tags: ','.join(tags),
    'DESC_SITUACAO':lambda tags: ','.join(tags),
    'ID_AUTOR': "first",
    'ID_PROPOSICAO':lambda tags: ','.join(tags),
    'PROPOSICAO_BASE':lambda tags: ','.join(tags),
    'URL_PROPOSICOES':lambda tags: ','.join(tags),
    'TEOR_URL':lambda tags: ','.join(tags)
})

# autores_G['ID_AUTOR'].replace('', np.nan, inplace=True)
# autores_G.reset_index(level=0, inplace=True)

# autores_G['index1'] = autores_G.index

# autores_G.reset_index(level=0, inplace=True)
# autores_G.shape[0]
# autores_G.head(4)

keepCol = ['NM_URNA','GENERO', 'COR_RACA', 'ESTADO_CIVIL','ESCOLARIDADE', 'OCUPACAO_ANTERIOR','FAIXA_ETARIA','PARTIDO_ATUAL', 'IDEOLOGIA_PARTIDO_ATUAL','MILIONARIO','ESTADO_ELEICAO','REGIAO_ELEICAO','ST_REELEICAO', 'AGRUPAMENTO_IDADE','AGRUPAMENTO_BENS','AGRUPAMENTO_VOTOS','AGRUPAMENTO_SEGUIDORES', 'AGRUPAMENTO_BALANCO_SEGUIDORES','AGRUPAMENTO_NUMERO_TWEETS', 'AGRUPAMENTO_BALANCO_TWEETS','ID_CAMARA','TEM_TWITTER','PERFIL_TWITTER','TOP_BTWEETS','TOP_BSEG','TOP_TWEETS','TOP_SEG','TOP_VOTACAO','TOP_BENS']

listaDeputados_F = listaDeputados[keepCol]

deputados_f = pd.DataFrame(listaDeputados_F[listaDeputados_F['TEM_TWITTER']=='S'])
# twitter_f.shape[0]

autores_proposicoes = pd.merge(autores_G.astype("str"), deputados_f.astype("str"), how='inner', left_on=['NOME_AUTOR'], right_on=['NM_URNA'])
# autores_proposicoes = autores_proposicoes.dropna()
# autores_proposicoes.shape[0]
# autores_proposicoes = autores_proposicoes.drop(['SG_PARTIDO', 'SG_UF'],axis=1)
# autores_proposicoes.head(3)
# autores_proposicoes.shape[0]
# autores_proposicoes.to_csv("autoresProposicoes.csv")


## 2.4 - Cria base de dados dos tweets

In [ ]:
tweets2020 = pd.DataFrame(tweetsDeputados[tweetsDeputados['coleta']=='C1'])
rt = ~tweets2020['text'].str.startswith('RT @')

tweets2020 = tweets2020[rt]

# list(tweetsDeputados.columns)
# tweetsDeputados.head(3)
tweets2020.to_csv("tweets2020.csv")

# csv.field_size_limit(sys.maxsize)
arq_csv = "tweets2020.csv"

f = open("stopwords.txt", 'r', encoding='utf-8')
stopwords = [name.rstrip().lower() for name in f]

with open(arq_csv, encoding="utf-8") as f:
    vTweets = [row["text"] for row in DictReader(f)]

vFrases = []

for idx,tweet in enumerate(vTweets):
    tweet = re.sub(r"https?:\/\/(\S+)", "", unidecode(tweet))
    # tira toda a pontuação exceto arroba e jogo da velha
    # tweet = re.sub(r"[^\P{P}@#]+", "", tweet)
    # tira toda a pontuação
    tweet = re.sub(r'[^\w\s]',' ',tweet)
    tweet = " ".join([x for x in tweet.split(' ') if x.lower() not in stopwords])
    tweet = tweet.rstrip() 
    tweet = tweet.strip() 
    tweet = tweet.lower()   
    vFrases.append(tweet) 
# vFrases

### 2.5 - Agrupa tweets com médias de rts e curtidas

In [ ]:
tweets2020['TWEET_KEYWORDS'] = vFrases
tweets2020['media_retweets'] = tweets2020['retweet_count']
tweets2020['soma_retweets'] = tweets2020['retweet_count']
tweets2020['media_fav'] = tweets2020['favorite_count']
tweets2020['soma_fav'] = tweets2020['favorite_count']

tweets2020['count'] = 1

# tweets2020.tail(3)

tweets2020_G = tweets2020.groupby(["screen_name"],as_index=False).agg({
    'id': "first",
    'screen_name': "first",
    'text':lambda tags: ','.join(tags),
    'TWEET_KEYWORDS':lambda tags: ','.join(tags),
    'count': 'sum',
    'media_retweets':'mean',
    'soma_retweets':'sum',
    'media_fav':'mean',
    'soma_fav':'sum',
    'created_at':lambda tags: ','.join(tags),
    'media_type':lambda tags: ','.join(tags),
    'lang':lambda tags: ','.join(tags),
    'coleta':'first'
})

tweets2020_G['media_retweets'] = tweets2020_G['media_retweets'].round(0)
tweets2020_G['media_fav'] = tweets2020_G['media_fav'].round(0)

# tweets2020_G.shape[0]
# tweets2020_G.head(3)



## 2.6 - Combina bancos de dados de proposicções e tweets e formata rótulos

In [ ]:
usuarios_proposicoes = pd.merge(autores_proposicoes,tweets2020_G, how='inner', left_on=['PERFIL_TWITTER'], right_on=['screen_name'])

# usuarios_proposicoes.reset_index(level=0, inplace=True)

usuarios_proposicoes = usuarios_proposicoes.rename(columns={'id':'TWEET_IDS','text':'TEXT', 'count':'TWEET_COUNTS', 'media_retweets':'MEDIA_RETWEETS', 'soma_retweets':'SOMA_RETWEETS', 'media_fav':'MEDIA_FAV', 'soma_fav':'SOMA_FAV', 'created_at':'CREATED_AT', 'media_type':'MEDIA_TYPE', 'lang':'LANG', 'coleta':'COLETA','screen_name':'SCREEN_NAME','PALAVRAS_CHAVE':'PLS_KEYWORDS'})

usuarios_proposicoes = usuarios_proposicoes.drop(['PERFIL_TWITTER'],axis=1)

# list(usuarios_proposicoes.columns)
# usuarios_proposicoes.shape[0]

# usuarios_proposicoes['PALAVRAS_CHAVE'] = usuarios_proposicoes['PALAVRAS_CHAVE'].astype("str").apply(lambda x: ','.join(set(x.split(','))))

# usuarios_proposicoes['TWEET_KEYWORDS'] = usuarios_proposicoes['TWEET_KEYWORDS'].astype("str").apply(lambda x: ','.join(set(x.split(','))))

usuarios_proposicoes['PLS_KEYWORDS'] = usuarios_proposicoes['PLS_KEYWORDS'].str.replace(",", " ")
usuarios_proposicoes['TWEET_KEYWORDS'] = usuarios_proposicoes['TWEET_KEYWORDS'].str.replace(",", " ")
# usuarios_proposicoes.head(3)

usuarios_proposicoes.to_csv("usuariosProposicoes.csv")


# usuarios_proposicoes = pd.read_csv("usuariosProposicoes.csv")
# usuarios_proposicoes.shape[0]

# usuarios_proposicoes.shape[0]
# usuarios_proposicoes = usuarios_proposicoes.dropna()
# usuarios_proposicoes.shape[0]

# usuarios_proposicoes.to_csv("usuariosProposicoes.csv")

# usuarios_proposicoes = usuarios_proposicoes.dropna()
# usuarios_proposicoes.shape[0]

## 2.7 - Gera gráfico da proporção de Deputados Federais que submeteram proposições e publicaram tweets em 2020

In [ ]:
top = percentual.plot(kind='barh', 
                        colormap='tab10', 
                        figsize=(11, 8),width=0.8)

for i, v in enumerate(list(percentual)):
    top.text(v+0.4, i-0.25, str(v)+"%" ,fontsize=12, color='black')

plt.tick_params(axis='y', which='major', labelsize=14)
    
# top.margins(x=1)
top.spines['right'].set_visible(False)
# top.spines['top'].set_visible(False)
# top.spines['bottom'].set_visible(False)
    
# top.get_legend().remove()            

# plt.rcParams.update(IPython_default);
# plt.style.use("fivethirtyeight")
plt.grid(color='b', linestyle='-', linewidth=.1)    
plt.savefig("novos-graficos/Figura x - Proporção de Deputados Federais que submeteram proposições e publicaram tweets em 2020.jpg",bbox_inches = 'tight')

## 2.8 - Cria índice de consistência temática

### 2.9 - Remove stop words das proposições </h3>

In [ ]:
keywordsCamara = usuarios_proposicoes['PLS_KEYWORDS'].astype("str").values.tolist()
# usuarios_proposicoes['PLS_KEYWORDS'].shape[0]
# len(keywordsCamara)
# len(keywordsTwitter)
# keywordsCamara[0]

f = open("stopwords.txt", 'r', encoding='utf-8')
stopwords = [name.rstrip().lower() for name in f]
vCamara = []
vCamara1 = []


for frase in keywordsCamara:
    frase = re.sub(r'[^\w\s]','',unidecode(frase))
    frase = re.sub(r'\n_',' ',frase)
    frase = re.sub(r'\n',' ',frase)
    frase = re.sub(r'_',' ',frase)
    # frase = re.sub(r' sobre | qualquer | das | da | dos | do | de | e | a | o | para | com | em | ou | à | na | no ',' ',frase)
    frase = " ".join([x for x in frase.split(' ') if x.lower() not in stopwords])
    frase = frase.replace(" do ","")
    frase = frase.lower()
    frase = frase.replace("    ","   ")
    frase = frase.replace("   ","  ")
    frase = frase.replace("  "," ")
    frase = frase.strip()
    vCamara1.append(frase)
    vfrases1 = frase.split(' ')
    vfrases2 = []
    for x in vfrases1:
        if x not in vfrases2 and x != '':
            vfrases2.append(x)
    vCamara.append(vfrases2)
    
    
# usuarios_proposicoes['PLS_KEYWORDS'].shape[0]
# len(vCamara)

# oe = 50

# len(vCamara[oe])
# vCamara[oe]
# len(vCamara1)

### 2.10 - Remove stop words dos tweets

In [ ]:
keywordsTwitter = usuarios_proposicoes['TWEET_KEYWORDS'].astype("str").values.tolist()

vTwitter = []

for frase in keywordsTwitter:
    # frase = re.sub(r'[^\w\s]','',frase)
    frase = re.sub(r'\n',' ',unidecode(frase))
    frase = frase.replace("   ","  ")
    frase = frase.replace("  "," ")
    # # frase = " ".join([x for x in tweet.split(' ') if x.lower() not in stopwords])
    # frase = frase.lower()
    frase = " ".join([x for x in frase.split(' ') if x.lower() not in stopwords])
    frase = frase.strip()
    # vTwitter.append(unidecode(frase).split(' '))
    vfrases1 = unidecode(frase).split(' ')
    vfrases2 = []
    for x in vfrases1:
        if x not in vfrases2:
            vfrases2.append(x)
    vTwitter.append(vfrases2)

    
    
# usuarios_proposicoes['TWEET_KEYWORDS'].head(3)
# usuarios_proposicoes['TWEET_KEYWORDS'].shape[0]
len(vTwitter[0])
# vTwitter[0]

# usuarios_proposicoes = pd.read_csv("usuariosProposicoes.csv")
# usuarios_proposicoes.to_csv("usuariosProposicoes.csv",index=False)

# usuarios_proposicoes = usuarios_proposicoes.dropna()
# usuarios_proposicoes.shape[0]

### 2.11 - Faz a interseção entre tweets e propostas

In [ ]:
somaLista = []
for x,y in zip(vCamara, vTwitter):
    somaLista.append(x+y)

# len(vTwitter[0])
# len(vCamara[0])
# len(somaLista[0])

uniaoLista = []

for lista in somaLista:
    vUnicos = []
    for palavra in lista:
        if palavra not in vUnicos:
            vUnicos.append(palavra)
    uniaoLista.append(vUnicos)
        
# somaLista[0]

### 2.12 - Mede a semelhança entre conteúdo dos tweets e das proposições e cria rótulos a partir dos resultados

In [ ]:
intersecao = [ (set.intersection(set(x), set(y))) for x,y in zip(vCamara, vTwitter)]
cTematica = [ (round(len(x)/len(y)*100,2)) for x,y in zip(intersecao,vCamara)]

# len(cTematica)
# usuarios_proposicoes.shape[0]

# teste1 = cTematica.dropna()
# len(teste1)

# print(type(cTematica))
# len(cTematica2)

usuarios_proposicoes['CONSISTENCIA_TEMATICA'] = cTematica
usuarios_proposicoes['INTERSECAO'] = intersecao
usuarios_proposicoes.to_csv("usuariosProposicoes.csv", index=False)
# teste = usuarios_proposicoes[['NM_URNA','CONSISTENCIA_TEMATICA']]
# teste.shape[0]
# teste.sort_values(by='CONSISTENCIA_TEMATICA', ascending=False)
# teste = teste.dropna()
# usuarios_proposicoes

faixa = ['cTEMATICA_G1','cTEMATICA_G2','cTEMATICA_G3','cTEMATICA_G4']
usuarios_proposicoes['AGRUPAMENTO_cTEMATICA'] = pd.qcut(usuarios_proposicoes['CONSISTENCIA_TEMATICA'],4,precision=0,labels=faixa)
# qcut = usuarios_proposicoes.groupby('AGRUPAMENTO_cTEMATICA').agg(lambda x: x.mode())
# qcut

# d2g.upload(usuarios_proposicoes, spreadsheet_key, 'usuariosProposicoes', credentials=credentials, row_names=True)

# quase tagcloud
# y = [(x, vCamara[0].count(x)) for x in set(vCamara[0])]

In [ ]:
# usuarios_proposicoes.columns
# usuarios_proposicoes2 = usuarios_proposicoes.drop(['TEXT','EMENTA','URL_PROPOSICOES','TEOR_URL','CREATED_AT','DATA_APRESENTACAO', 'DESC_TRAMITACAO', 'DESC_SITUACAO','ID_AUTOR', 'ID_PROPOSICAO', 'PROPOSICAO_BASE','PLS_KEYWORDS','TWEET_KEYWORDS','NM_URNA'], axis=1)
# d2g.upload(usuarios_proposicoes2, spreadsheet_key, 'usuariosProposicoes', credentials=credentials, row_names=True)

### 2.13 - Calcula a média geral de consistência temática

In [ ]:
ct_genero = usuarios_proposicoes.groupby(["GENERO"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_genero['CATEGORIA'] = "GENERO"
ct_raca = usuarios_proposicoes.groupby(["COR_RACA"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_raca['CATEGORIA'] = "COR_RACA"
ct_estadoCivil = usuarios_proposicoes.groupby(["ESTADO_CIVIL"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_estadoCivil['CATEGORIA'] = "ESTADO_CIVIL"
ct_escolaridade = usuarios_proposicoes.groupby(["ESCOLARIDADE"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_escolaridade['CATEGORIA'] = "ESCOLARIDADE"
ct_ocupacao = usuarios_proposicoes.groupby(["OCUPACAO_ANTERIOR"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ocupacao['CATEGORIA'] = "OCUPACAO_ANTERIOR"
ct_partido = usuarios_proposicoes.groupby(["PARTIDO_ATUAL"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_partido['CATEGORIA'] = "PARTIDO_ATUAL"
ct_ideologia = usuarios_proposicoes.groupby(["IDEOLOGIA_PARTIDO_ATUAL"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ideologia['CATEGORIA'] = "IDEOLOGIA_PARTIDO_ATUAL"
ct_milionario = usuarios_proposicoes.groupby(["MILIONARIO"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_milionario['CATEGORIA'] = "MILIONARIO"
ct_estado = usuarios_proposicoes.groupby(["ESTADO_ELEICAO"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_estado['CATEGORIA'] = "ESTADO_ELEICAO"
ct_regiao = usuarios_proposicoes.groupby(["REGIAO_ELEICAO"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_regiao['CATEGORIA'] = "REGIAO_ELEICAO"
ct_reeleito = usuarios_proposicoes.groupby(["ST_REELEICAO"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_reeleito['CATEGORIA'] = "ST_REELEICAO"
ct_ag_idade = usuarios_proposicoes.groupby(["AGRUPAMENTO_IDADE"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ag_idade['CATEGORIA'] = "AGRUPAMENTO_IDADE"
ct_ag_bens = usuarios_proposicoes.groupby(["AGRUPAMENTO_BENS"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ag_bens['CATEGORIA'] = "AGRUPAMENTO_BENS"
ct_ag_votos = usuarios_proposicoes.groupby(["AGRUPAMENTO_VOTOS"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ag_votos['CATEGORIA'] = "AGRUPAMENTO_VOTOS"
ct_ag_seguidores = usuarios_proposicoes.groupby(["AGRUPAMENTO_SEGUIDORES"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ag_seguidores['CATEGORIA'] = "AGRUPAMENTO_SEGUIDORES"
ct_ag_bseguidores = usuarios_proposicoes.groupby(["AGRUPAMENTO_BALANCO_SEGUIDORES"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ag_bseguidores['CATEGORIA'] = "AGRUPAMENTO_BALANCO_SEGUIDORES"
ct_ag_tweets = usuarios_proposicoes.groupby(["AGRUPAMENTO_NUMERO_TWEETS"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ag_tweets['CATEGORIA'] = "AGRUPAMENTO_NUMERO_TWEETS"
ct_ag_btweets = usuarios_proposicoes.groupby(["AGRUPAMENTO_BALANCO_TWEETS"]).agg({'CONSISTENCIA_TEMATICA':'mean'})
ct_ag_btweets['CATEGORIA'] = "AGRUPAMENTO_BALANCO_TWEETS"

In [ ]:
consistencia_tematica = pd.concat([ct_genero, ct_raca, ct_estadoCivil, ct_escolaridade, ct_partido, ct_ideologia, ct_milionario, ct_estado, ct_regiao, ct_reeleito, ct_ag_idade, ct_ag_bens, ct_ag_votos, ct_ag_seguidores, ct_ag_bseguidores, ct_ag_tweets, ct_ag_btweets, ct_ocupacao])

consistencia_tematica['CONSISTENCIA_TEMATICA'].mean()
# consistencia_tematica

### 2.14 - Calcula a média de consistência temática por estado

In [ ]:
ct_estado = usuarios_proposicoes.groupby(["ESTADO_ELEICAO"]).agg({'CONSISTENCIA_TEMATICA':'mean'})

ct_estado['CONSISTENCIA_TEMATICA'] = ct_estado['CONSISTENCIA_TEMATICA'].round(2)

ct_estado['CONSISTENCIA_TEMATICA'].mean()


# ct_estado
# print(type(ct_estado))

In [ ]:
# list(usuarios_proposicoes.columns)
usuarios_proposicoes['N_PLS']

### 2.15 - Gera gráfico com número de tweets por Estado

In [ ]:
t_estado = usuarios_proposicoes.groupby(["ESTADO_ELEICAO"]).agg({'TWEET_COUNTS':'sum'})

t_estado = t_estado.sort_values(by='TWEET_COUNTS', ascending=True)

total = t_estado['TWEET_COUNTS']
total

# ct_estado
# print(type(ct_estado))

In [ ]:
                        
top = total.plot(kind='barh', 
                        colormap='tab10', 
                        figsize=(12, 8),width=0.8)

for i, v in enumerate(list(total)):
    top.text(v+0.4, i-0.25, v ,fontsize=12, color='black')

plt.tick_params(axis='y', which='major', labelsize=14)
    
# top.margins(x=1)
# top.spines['right'].set_visible(False)
# top.spines['top'].set_visible(False)
# top.spines['bottom'].set_visible(False)
    
# top.get_legend().remove()            
top.set_ylabel('')
# plt.rcParams.update(IPython_default);
# plt.style.use("fivethirtyeight")
plt.grid(color='b', linestyle='-', linewidth=.1)    
plt.savefig("novos-graficos/Figura x - Quantidade de Tweets por Estado.jpg",bbox_inches = 'tight')
# plt.show()

### 2.16 - Cria tabela com as médias de consistência temática por estado

In [ ]:


faixa = ['G4','G3','G2','G1']
ct_estado['AGRUPAMENTO'] = pd.qcut(ct_estado['CONSISTENCIA_TEMATICA'],4,precision=0,labels=faixa)


ct_estado = ct_estado.sort_values(['CONSISTENCIA_TEMATICA'], ascending=[False])


ct_estado.to_csv("consistenciaTematica-estados.csv", index=False)

ct_estado
# qcut = usuarios_proposicoes.groupby('AGRUPAMENTO_cTEMATICA').agg(lambda x: x.mode())
# qcut

# d2g.upload(ct_estado, spreadsheet_key, 'consistenciaTematica', credentials=credentials, row_names=True)


### 2.17 - Salva resultados em arquivo csv

In [ ]:
%%capture

# usuarios_proposicoes.shape[0]
# len(vCamara)
# len(vTwitter)

usuarios_proposicoes['PLS_KEYWORDS'] = vCamara
usuarios_proposicoes['TWEET_KEYWORDS'] = vTwitter


usuarios_proposicoes['PLS_KEYWORDS'] = usuarios_proposicoes['PLS_KEYWORDS'].astype(str).str.replace(',',' ')
usuarios_proposicoes['PLS_KEYWORDS'] = usuarios_proposicoes['PLS_KEYWORDS'].astype(str).str.replace('[^\w\s]','')



usuarios_proposicoes['TWEET_KEYWORDS'] = usuarios_proposicoes['TWEET_KEYWORDS'].astype(str).str.replace(',',' ')
usuarios_proposicoes['TWEET_KEYWORDS'] = usuarios_proposicoes['TWEET_KEYWORDS'].astype(str).str.replace('[^\w\s]','')


usuarios_proposicoes['INTERSECAO'] = usuarios_proposicoes['INTERSECAO'].astype(str).str.replace(',',' ')
usuarios_proposicoes['INTERSECAO'] = usuarios_proposicoes['INTERSECAO'].astype(str).str.replace('[^\w\s]','')



# usuarios_proposicoes.head(2)

usuarios_proposicoes.to_csv("usuarios_proposicoes-2.csv", index=False)


## 2.18 - Gera gráficos gerais

### 2.19 - Maiores e menores índices de consistência temática

In [ ]:
ct_estado = ct_estado.drop('AGRUPAMENTO', 1)
ct_estado

In [ ]:
ptop10 = ct_estado.nlargest(12,'CONSISTENCIA_TEMATICA').sort_values(by='CONSISTENCIA_TEMATICA', ascending=True)
indices1  = ptop10.apply(lambda row: str(row.CONSISTENCIA_TEMATICA) +"%", axis=1)
indices1 = list(indices1)
ptop10 = ptop10['CONSISTENCIA_TEMATICA']

pbottom10 = ct_estado.nsmallest(15,'CONSISTENCIA_TEMATICA').sort_values(by='CONSISTENCIA_TEMATICA', ascending=True)

indices2  = pbottom10.apply(lambda row: str(row.CONSISTENCIA_TEMATICA) +"%", axis=1)
# indices2 = list(pbottom10['CONSISTENCIA_TEMATICA'])
indices2 = list(indices2)
pbottom10 = pbottom10['CONSISTENCIA_TEMATICA']

In [ ]:
axd = plt.figure(figsize=(8,10)).subplot_mosaic(
    """
    A
    B
    """
)

top10 = ptop10.plot(kind='barh', 
                        color='#0A8CB1', 
                        width=0.8,ax=axd['A'])


for i, v in enumerate(list(ptop10)):
    top10.text(v-5, i-0.1, indices1[i], color='white',weight='heavy')
    
    

    
# for i, v in enumerate(list(votacoes_10)):
#     top.text(v-20, i-0.15, indices1[i] ,fontsize=12, color='white',weight='heavy')



# for i, v in enumerate(list(total)):
#     top.text(v+0.4, i-0.25, v ,fontsize=12, color='black')

# top10 = sns.barplot(color='#0A8CB1',data=ptop10,ax=axd['A'])
# top10 = sns.barplot(color='#0A8CB1',data=ptop10)


bottom10 = pbottom10.plot(kind='barh', 
                        color='#BC0D0D', 
                        width=0.8,ax=axd['B'])

# bottom10 = sns.barplot(color='#BC0D0D',data=pbottom10,ax=axd['B'])

for i, v in enumerate(list(pbottom10)):
    bottom10.text(v-4, i-0.1, indices2[i], color='white',weight='heavy')
    
    
# for i, v in enumerate(list(votacoes_10)):
#     top.text(v-20, i-0.15, indices[i] ,fontsize=12, color='white',weight='heavy')

# top10.get_legend().remove()  
# bottom10.get_legend().remove()  

top10.set_ylabel('')    
top10.set_xlabel('')

bottom10.set_ylabel('')    
bottom10.set_xlabel('')

bottom10.grid(color='b', linestyle='-', linewidth=.1)   
top10.grid(color='b', linestyle='-', linewidth=.1)   

plt.tight_layout()
# axd

# plt.show()

plt.savefig("novos-graficos/Figura x - Índices de consistência temática por estado.jpg",bbox_inches = 'tight')
plt.clf()
# plt.show()

### 2.20 - Temas e palavras-chave mais frequentes nas proposições

In [ ]:
vCamara2 = [item for sublist in vCamara for item in sublist]
vCamara3 = ' '.join([str(elem) for elem in vCamara2])

# if "codigo" in vCamara2:
#     print("é ta sim")
# else:
#     print("ta nao")

# len(vCamara2)
# len(vCamara1)

axd = plt.figure(figsize=(16,8)).subplot_mosaic(
    """
    AB
    """, 
    gridspec_kw={
        'width_ratios': [5, 6],
        'wspace': 0.1
    })

lang_responses = usuarios_proposicoes['TEMA']

language_counter = Counter()
    
for response in lang_responses:
    language_counter.update(response.split(','))
        
languages = []
popularity = []
    
for item in language_counter.most_common(10):
    languages.append(item[0])
    popularity.append(item[1])

# n30 = popularity[:10]
# n30 = popularity[-30:]


languages.reverse()
popularity.reverse()


axd['A'].barh(languages, popularity)

# axd['A'].yticks(fontsize=28)
axd['A'].set_yticklabels(languages,fontsize=18)

for i, v in enumerate(popularity):
    axd['A'].text(v-200, i-.07, str(v), color='white',size=17)

# --------------------------------------------------------------------------


wordcloud = WordCloud(
    # max_font_size=80,
    min_font_size=22,
    max_words=15,
    prefer_horizontal=1,
    # scale=10,
    height=640,
    width=550,
    background_color = 'white',
    stopwords=stopwords).generate(str(vCamara3))

# axd['A'].set_title('Temas',fontsize=20)
# axd['B'].set_title('Palavras-chave',fontsize=20)

axd['B'].axis('off')
axd['B'].imshow(wordcloud, interpolation = 'bilinear')



# plt.set_title('Palavras-chave',fontsize=20)
# plt.axis('off')
# plt.imshow(wordcloud, interpolation = 'bilinear')

# plt.tight_layout(pad=0)
# plt.show()


plt.savefig("2 - Figura 8 - Temas e palavras-chave mais recorrentes nas proposições.jpg",bbox_inches = 'tight')
# plt.figure(figsize=(5,10))
# bottom10 = sns.barplot(x=popularityr,y=languagesr, color='#0A8CB1')
    

    
# plt.barh(languages, popularity)
# # plt.title("Most popular languages")
# # plt.ylabel("Programming Languages")
# # plt.xlabel("Number of People who use")

# plt.tight_layout()
# plt.show()

# quase tagcloud
# y = [(x, vCamera.count(x)) for x in set(vCamera)]

### 2.21 - Publicações mais compartilhadas no Twitter

In [ ]:

# tweets2020.head(2)
topTweets = tweets2020.nlargest(3,'retweet_count')
# tweetsTop10
topIds = topTweets['id'].astype("str").values.tolist()

string = "https://twitter.com/OReillyMedia/status/"

topIds = [string + s for s in topIds]
# topIds

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text

# Tweet("https://twitter.com/OReillyMedia/status/1253710046405500928")
    
for x in topIds:
    Tweet(x)
# vTweets(topIds)
    

<h3> 2.22 - Usuários, hashtags e palavras mais utlizadas no Twitter </h3>

In [ ]:
keywordsTwitter = usuarios_proposicoes['TWEET_KEYWORDS'].astype("str").values.tolist()

vTwitter1 = []

f = open("stopwords.txt", 'r', encoding='utf-8')
stopwords = [name.rstrip().lower() for name in f]

for tweet in keywordsTwitter:
    tweet = re.sub(r'\n',' ',unidecode(tweet))
    tweet = tweet.replace("  "," ")
    # tweet = tweet.replace("paraben","parabens")
    tweet = " ".join([x for x in tweet.split(' ') if x.lower() not in stopwords])
    vTwitter1.append(tweet) 
# vFrases  

# tweets2020['TWEET_KEYWORDS'] = vFrases
# len(vTwitter)
# len(vTwitter1)
# vTwitter[:10]

#### 2.23 - Extrai as informações dos tweets

In [ ]:
words_repetition = 1
hashtags_repetition = 1
users_repetition = 1

#variables for how many words, hashtags, users to display
word_limit = 10
hashtag_limit = 10
user_limit = 10

#file path for stopwords and json exports
stopwords_file_path = 'stopwords.txt'
# json_export_file_path = '1-total.json'

with open(stopwords_file_path, encoding='utf-8') as f:
    stopwords = f.read().splitlines()

items = tweets2020.to_dict('records')
# items[0]

#taken from https://stackoverflow.com/a/49146722
def remove_emoji(string):
    emoji_pattern = re.compile("["
                        "\U0001F600-\U0001F64F"  # emoticons
                        "\U0001F300-\U0001F5FF"  # symbols & pictographs
                        "\U0001F680-\U0001F6FF"  # transport & map symbols
                        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        "\U00002702-\U000027B0"
                        "\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


#taken from https://gist.github.com/glenbot/4684356
def sanitize_3(user_input, stop_words):
    """Sanitize using standard lists"""
    new_list = []
    for w in user_input:
        if w not in stop_words:
            new_list.append(w)
    return new_list


#remove accents, emoji, convert to all lowercase
for idx, item in enumerate(items):
    items[idx]['text'] = remove_emoji(items[idx]['text'])
    items[idx]['text'] = unidecode(items[idx]['text'])
    items[idx]['text'] = items[idx]['text'].lower()
    #print items[idx]['text']


#removing stopwords
#convert every tweet into a list of words
#call sanitize_3 for every list of words
current_string = ''
placeholder_list = [None] * len(items)
master_word_list = []
master_hashtag_list = []
master_user_list = []

for idx, item in enumerate(items):
    #word_list = re.sub("[^\w]", " ", items[idx]['text']).split()
    word_list = re.sub(r'[.!,;:?]', ' ', items[idx]['text']).split()
    word_list = sanitize_3(word_list, stopwords)
    for idx2, word in enumerate(word_list):
        if word_list[idx2][:1] == '#':
            current_string += ' ' + word_list[idx2]
            master_hashtag_list.append(word_list[idx2])
        elif word_list[idx2][:1] == '@':
            current_string += ' ' + word_list[idx2]
            master_user_list.append(word_list[idx2])
        elif word_list[idx2][:3] == '//t':              #dirty workaround
            pass
        elif 'co/' not in word_list[idx2]:
            current_string += ' ' + word_list[idx2]
            master_word_list.append(word_list[idx2])
    placeholder_list[idx] = current_string
    current_string = ''


#release memory immediately
del placeholder_list[:]

#rank words
#adapted from https://github.com/kevinschaul/Word-Rank/blob/master/wordRank.py
l = {}
hashtag_dict = {}
user_dict = {}

for word in master_word_list:
    # if word is in dictionary, increment the value
    # otherwise add the word to dictionary with value 1
    if word in l:
        l[word] += 1
    else:
        l[word] = 1

for hashtag in master_hashtag_list:
    if hashtag in hashtag_dict:
        hashtag_dict[hashtag] += 1
    else:
        hashtag_dict[hashtag] = 1

for user in master_user_list:
    if user in user_dict:
        user_dict[user] += 1
    else:
        user_dict[user] = 1


wordsV1 = []
wordsV2 = []

# this prints the dict out d by value in descending order
count = 0
# print('\n#########   Printing ranking of words   #########\n')
for key, value in sorted(iter(l.items()), reverse=True, key=lambda k_v3: (k_v3[1],k_v3[0])):
    if count < word_limit:
        # print('%s: %s' % (key, value))
        wordsV1.append(key)
        wordsV2.append(value)
    count+=1

wordsD1 = dict(zip(wordsV1, wordsV2))
# print(wordsD1)

hashtagV1 = []
hashtagV2 = []

count = 0
# print('\n#########   Printing ranking of hashtags   #########\n')
for key, value in sorted(iter(hashtag_dict.items()), reverse=True, key=lambda k_v4: (k_v4[1],k_v4[0])):
    if count < hashtag_limit:
        # print('%s: %s' % (key, value))
        hashtagV1.append(key)
        hashtagV2.append(value)
        
    count +=1

hashtagD1 = dict(zip(hashtagV1, hashtagV2))

# print(hashtagD1)

# plt.rcdefaults()
# fig, ax = plt.subplots()

# y_pos = np.arange(len(hashtagV1))
# error = np.random.rand(len(hashtagV1))

# ax.barh(y_pos, hashtagV2, xerr=error, align='center')
# ax.set_yticks(y_pos)
# ax.set_yticklabels(hashtagV1)
# ax.invert_yaxis()  # labels read top-to-bottom
# ax.set_xlabel('Frequência')
# ax.set_title('Hashtags mais utilizadas')

# plt.show()


usersV1 = []
usersV2 = []

count = 0
# print('\n#########   Printing ranking of most mentioned users   #########\n')
for key, value in sorted(iter(user_dict.items()), reverse=True, key=lambda k_v5: (k_v5[1],k_v5[0])):
    if count < user_limit:
        # print('%s: %s' % (key, value))
        usersV1.append(key)
        usersV2.append(value)
    count += 1


usersD1 = dict(zip(usersV1, usersV2))
# print(usersD1)

most_retweeted_list = []
list_is_full = False

for tweet in items:
    if 'rt @' in tweet['text']:
        pass
    elif tweet['retweet_count'] > 10:
        most_retweeted_list.append(tweet)

count = 0


tweets =[]
rts = []
ids = []

# print('\n#########   Printing most retweeted tweets   #########\n')
for tweet in sorted(most_retweeted_list, key = lambda x: x['retweet_count'], reverse=True):
    current_date = datetime.strptime(tweet['created_at'],'%Y-%m-%d %H:%M:%S+00:00')
    # if (current_date.year == 2020) and (current_date.month == 11) and (current_date.day < 17):
    tweets.append(tweet['screen_name'] +" - "+ tweet['text'])
    rts.append(tweet['retweet_count'])
    ids.append(tweet['id'])
    if count == 20:
        break
    count+=1

# rtKeys1 = ['tweet','rts','ids']
# rtKeys2 =  rtKeys1 *len(ids)
# print(rtKeys2)

retweetsD1 = [{'tweet': tweet, 'rts': rt, 'ids': idd} for tweet,rt,idd in zip(tweets,rts,ids)]



day_list = []
count = 0
for tweet in items:
    current_date = datetime.strptime(tweet['created_at'],'%Y-%m-%d %H:%M:%S+00:00')
    # if (current_date.year == 2020) and (current_date.month == 11) and (current_date.day < 17):
    date_time_str = str(current_date.day) +"/"+ str(current_date.month) +"/"+ str(current_date.year)
    # print(date_time_str)
    # day_list.append(str(current_date.day) +"/"+ str(current_date.month))
    date_time_obj = datetime.strptime(date_time_str, '%d/%m/%Y')
    # print(date_time_obj)
    day_list.append(date_time_obj)

day_dict = {}


for day in day_list:
    if day in day_dict:
        day_dict[day] += 1
    else:
        day_dict[day] = 1


DatasV1 = []
DatasV2 = []

# #going to break on cases that go from 23 to midnight or backwards
# print('\n#########   Printing hourly traffic   #########\n')
for key, value in sorted(iter(day_dict.items()), reverse=False, key=lambda k_v: (k_v[0],k_v[1])):
    chave = str(key)
    chave = re.sub(" 00:00:00","", chave)
    DatasV1.append(chave)
    DatasV2.append(value)
    # print('%s: %s' % (key, value))
DatasD1 = dict(zip(DatasV1, DatasV2))

# json.dump({report:[{'words':wordsD1},{'hashtags':hashtagD1},{'users':usersD1},{'retweets':retweetsD1},{'dates':DatasD1}]},pre_jsonReport, indent=4)

    

#### 2.24 - Gera a visualização

In [ ]:
axd = plt.figure(figsize=(7,15)).subplot_mosaic(
    """
    A
    B
    C
    """,
    gridspec_kw={
        # 'width_ratios': [5, 5, 7],
        # 'height_ratios': [5, 5, 7],
        # 'right': 1.2,
        # 'left': 1,
        'hspace': 0.3
        
    })





# usersV1.reverse()
# usersV2.reverse()

# hashtagV1.reverse()
# hashtagV2.reverse()


axd['A'].barh(usersV1, usersV2, color='green')

for i, v in enumerate(usersV2):
    axd['A'].text(v-450, i-.07, str(v), color='white',size=12)



axd['B'].barh(hashtagV1, hashtagV2, color='purple')

for i, v in enumerate(hashtagV2):
    axd['B'].text(v-200, i-0.1, str(v), color='white',size=12)


axd['A'].set_yticklabels(usersV1,fontsize=13)
axd['B'].set_yticklabels(hashtagV1,fontsize=13)

   
# --------------------------------------------------------------------------

wordcloud = WordCloud(
    # width = 280,
    # height = 2800,
    prefer_horizontal=1,
    max_font_size=40,
    min_font_size=10,
    max_words=18,
    background_color = 'white',
    color_func=lambda *args, **kwargs: "#D38B07",
    stopwords=stopwords).generate(str(vTwitter1))



axd['A'].set_title('Usuários',fontsize=16)
axd['B'].set_title('Hashtags',fontsize=16)
axd['C'].set_title('Palavras',fontsize=16)

axd['C'].axis('off')
axd['C'].imshow(wordcloud)




# axd['A'].add_gridspec={'right': 100,'wspace': 30}
plt.tight_layout()

# axd['C'].imshow(wordcloud, interpolation = 'bilinear')


plt.savefig("4 - Figura 10 - Usuários, hashtags e palavras mais utlizadas no Twitter.jpg",bbox_inches = 'tight')



## 2.25 - Gera gráficos do Rio Grande do Sul

### 2.26 - Carrega base de dados

In [ ]:
usuarios_proposicoes = pd.read_csv("usuarios_proposicoes-2.csv")
tweets2020 = pd.read_csv("tweets2020.csv")
# usuarios_proposicoes.head(2)
usuarios_proposicoes_rs = usuarios_proposicoes[(usuarios_proposicoes['ESTADO_ELEICAO']=='RS')]

### 2.27 - Temas mais recorrentes nas proposições

In [ ]:
%%capture

perfil = 'ESTADO_ELEICAO'

filtro =usuarios_proposicoes[usuarios_proposicoes['ESTADO_ELEICAO'].isin(['RS'])]

lang_responses = usuarios_proposicoes['TEMA']

language_counter = Counter()
    
for response in lang_responses:
    language_counter.update(response.split(','))
        
languages = []
popularity = []
    
for item in language_counter.most_common(100):
    languages.append(item[0])
    popularity.append(item[1])
    
    

# language_counter



contagem_geral = dict(language_counter)
df = pd.DataFrame.from_dict(contagem_geral, orient='index')
# df.columns['TEMA','COUNT']
           
     
           
df.rename(columns = {0:'count'},inplace = True)

df.index.name = 'TEMA'
df['ESTADO_ELEICAO'] = 'GERAL'


df.reset_index()
df.columns
# df
temas = df[['ESTADO_ELEICAO', 'count']]
temas = temas.reset_index()

In [ ]:
soma = temas.groupby([perfil],as_index=False).agg({
    perfil: "first",
    'count': 'sum'
})



temasM_geral = pd.merge(temas, soma, how="left", on=perfil)
temasM_geral['proporcao'] = round(temasM_geral['count_x']/temasM_geral['count_y']*100,2)


# rd = px.line_polar(temasM_geral, r="proporcao", theta="TEMA", color=perfil,line_close=True, width=wd, height=ht)

# rd.update_layout(
#     margin=dict(l=ll, r=rr, t=tt, b=bb),

# )

In [ ]:
temasM_geral

In [ ]:
%%capture
perfil = 'ESTADO_ELEICAO' 
filtro =usuarios_proposicoes[usuarios_proposicoes['ESTADO_ELEICAO'].isin(['RS'])]



filtro =usuarios_proposicoes[usuarios_proposicoes['ESTADO_ELEICAO'].isin(['RS'])]

lang_responses = filtro['TEMA']

language_counter = Counter()
    
for response in lang_responses:
    language_counter.update(response.split(','))
        
languages = []
popularity = []
    
for item in language_counter.most_common(100):
    languages.append(item[0])
    popularity.append(item[1])
    
    

# language_counter



contagem_geral = dict(language_counter)
df = pd.DataFrame.from_dict(contagem_geral, orient='index')
# df.columns['TEMA','COUNT']
           
     
           
df.rename(columns = {0:'count'},inplace = True)

df.index.name = 'TEMA'
df['ESTADO_ELEICAO'] = 'RS'


df.reset_index()
df.columns
# df
temas = df[['ESTADO_ELEICAO', 'count']]
temas = temas.reset_index()

In [ ]:
soma = temas.groupby([perfil],as_index=False).agg({
    perfil: "first",
    'count': 'sum'
})



temasM_rs = pd.merge(temas, soma, how="left", on=perfil)
temasM_rs['proporcao'] = round(temasM_rs['count_x']/temasM_rs['count_y']*100,2)

In [ ]:
temasM_rs.sort_values(by='count_x', ascending=False)

In [ ]:
temasM_geral = temasM_geral.nlargest(30, 'count_x')
temasM_geral = temasM_geral.sort_values(by='TEMA')

temasM_rs = temasM_rs.nlargest(30, 'count_x')
temasM_rs = temasM_rs.sort_values(by='TEMA')

TEMAS_C = pd.concat([temasM_geral, temasM_rs])
TEMAS_C.to_clipboard()

In [ ]:
wd = 1000
ht = 800
ll = 300
rr = 250
tt = 5
bb = 5

arq = "novos-graficos/Gráfico x - Temas mais recorrentes nas proposições - Rio Grande do Sul.jpg"

rd = px.line_polar(TEMAS_C, r="proporcao", theta="TEMA", color=perfil,line_close=True, width=wd, height=ht)

rd.update_layout(
    margin=dict(l=ll, r=rr, t=tt, b=bb),

)

# rd.show()

rd.write_image(arq)

### 2.28 - Publicações mais compartilhadas no Twitter

In [ ]:
from IPython.core.display import display, HTML
# perfil = 'ST_REELEICAO'
def topRTs2(perfil):
    unik = list(filtro[perfil].unique())
    # len(unik)
    # listalista = [[] for i in range(len(unik))]
    listas = []
    # lista


    for idx, k in enumerate(unik):
        # listalista[idx].append(filtro[filtro[perfil]==k]['SCREEN_NAME'].astype("str").values.tolist())
        listas.append(filtro[filtro[perfil]==k]['SCREEN_NAME'].astype("str").values.tolist())

    # len(lista)
    # lista[1]


    for idx,s in enumerate(listas):
        print(unik[idx])
        tweets2020F = tweets2020[tweets2020['screen_name'].isin(listas[idx])]        
        # tweets2020F

        # 
        # tweets2020.head(2)
        topTweets = tweets2020F.nlargest(5,'retweet_count')
        # tweetsTop10
        topIds = topTweets['id'].astype("str").values.tolist()

        string = "https://twitter.com/OReillyMedia/status/"

        topIds = [string + s for s in topIds]
        # topIds

        class Tweet(object):
            def __init__(self, s, embed_str=False):
                try:
                    if not embed_str:
                        # Use Twitter's oEmbed API
                        # https://dev.twitter.com/web/embedded-tweets
                        api = 'https://publish.twitter.com/oembed?url={}'.format(s)
                        response = requests.get(api)
                        self.text = response.json()["html"]
                        display(HTML(self.text))
                    else:
                        self.text = s
                except Exception:
                    pass
                        

            def _repr_html_(self):
                try:
                    return self.text
                except Exception:
                    pass

        # Tweet("https://twitter.com/OReillyMedia/status/1253710046405500928")

        for x in topIds:
            Tweet(x)
        # vTweets(topIds)
        


In [ ]:
perfil = 'ESTADO_ELEICAO' 
filtro =usuarios_proposicoes[usuarios_proposicoes['ESTADO_ELEICAO'].isin(['RS'])]
topRTs2(perfil)

<h3>2.29 - Usuários e hashtags e palavras mais frequentes no Twitter </h3>

In [ ]:
usuarios_proposicoes_rs = usuarios_proposicoes[(usuarios_proposicoes['ESTADO_ELEICAO']=='RS')]

In [ ]:
listaDeputados = pd.read_csv("db-listaDeputados.csv")

In [ ]:
keywordsTwitter = usuarios_proposicoes_rs['TWEET_KEYWORDS'].astype("str").values.tolist()

vTwitter1 = []

f = open("stopwords.txt", 'r', encoding='utf-8')
stopwords = [name.rstrip().lower() for name in f]

for tweet in keywordsTwitter:
    tweet = re.sub(r'\n',' ',unidecode(tweet))
    tweet = tweet.replace("  "," ")
    # tweet = tweet.replace("paraben","parabens")
    tweet = " ".join([x for x in tweet.split(' ') if x.lower() not in stopwords])
    vTwitter1.append(tweet) 
# vFrases  

# tweets2020['TWEET_KEYWORDS'] = vFrases
# len(vTwitter)
# len(vTwitter1)
# vTwitter[:10]

#### 2.30 - Extrai as informações dos tweets

In [ ]:
listaDeputados.columns

In [ ]:
# len(tweets2020)

tweets2020_info = tweets2020.merge(listaDeputados, left_on='screen_name', right_on='PERFIL_TWITTER')
# len(tweets2020_info)
# tweets2020_rs = 
# tweets2020_info

tweets2020_info_rs = tweets2020_info[(tweets2020_info['ESTADO_ELEICAO']=='RS')]
len(tweets2020_info_rs)


In [ ]:
words_repetition = 1
hashtags_repetition = 1
users_repetition = 1

#variables for how many words, hashtags, users to display
word_limit = 10
hashtag_limit = 10
user_limit = 10

#file path for stopwords and json exports
stopwords_file_path = 'stopwords.txt'
# json_export_file_path = '1-total.json'

with open(stopwords_file_path, encoding='utf-8') as f:
    stopwords = f.read().splitlines()

items = tweets2020_info_rs.to_dict('records')
# items[0]

#taken from https://stackoverflow.com/a/49146722
def remove_emoji(string):
    emoji_pattern = re.compile("["
                        "\U0001F600-\U0001F64F"  # emoticons
                        "\U0001F300-\U0001F5FF"  # symbols & pictographs
                        "\U0001F680-\U0001F6FF"  # transport & map symbols
                        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        "\U00002702-\U000027B0"
                        "\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


#taken from https://gist.github.com/glenbot/4684356
def sanitize_3(user_input, stop_words):
    """Sanitize using standard lists"""
    new_list = []
    for w in user_input:
        if w not in stop_words:
            new_list.append(w)
    return new_list


#remove accents, emoji, convert to all lowercase
for idx, item in enumerate(items):
    items[idx]['text'] = remove_emoji(items[idx]['text'])
    items[idx]['text'] = unidecode(items[idx]['text'])
    items[idx]['text'] = items[idx]['text'].lower()
    #print items[idx]['text']


#removing stopwords
#convert every tweet into a list of words
#call sanitize_3 for every list of words
current_string = ''
placeholder_list = [None] * len(items)
master_word_list = []
master_hashtag_list = []
master_user_list = []

for idx, item in enumerate(items):
    #word_list = re.sub("[^\w]", " ", items[idx]['text']).split()
    word_list = re.sub(r'[.!,;:?]', ' ', items[idx]['text']).split()
    word_list = sanitize_3(word_list, stopwords)
    for idx2, word in enumerate(word_list):
        if word_list[idx2][:1] == '#':
            current_string += ' ' + word_list[idx2]
            master_hashtag_list.append(word_list[idx2])
        elif word_list[idx2][:1] == '@':
            current_string += ' ' + word_list[idx2]
            master_user_list.append(word_list[idx2])
        elif word_list[idx2][:3] == '//t':              #dirty workaround
            pass
        elif 'co/' not in word_list[idx2]:
            current_string += ' ' + word_list[idx2]
            master_word_list.append(word_list[idx2])
    placeholder_list[idx] = current_string
    current_string = ''


#release memory immediately
del placeholder_list[:]

#rank words
#adapted from https://github.com/kevinschaul/Word-Rank/blob/master/wordRank.py
l = {}
hashtag_dict = {}
user_dict = {}

for word in master_word_list:
    # if word is in dictionary, increment the value
    # otherwise add the word to dictionary with value 1
    if word in l:
        l[word] += 1
    else:
        l[word] = 1

for hashtag in master_hashtag_list:
    if hashtag in hashtag_dict:
        hashtag_dict[hashtag] += 1
    else:
        hashtag_dict[hashtag] = 1

for user in master_user_list:
    if user in user_dict:
        user_dict[user] += 1
    else:
        user_dict[user] = 1


wordsV1 = []
wordsV2 = []

# this prints the dict out sorted by value in descending order
count = 0
# print('\n#########   Printing ranking of words   #########\n')
for key, value in sorted(iter(l.items()), reverse=True, key=lambda k_v3: (k_v3[1],k_v3[0])):
    if count < word_limit:
        # print('%s: %s' % (key, value))
        wordsV1.append(key)
        wordsV2.append(value)
    count+=1

wordsD1 = dict(zip(wordsV1, wordsV2))
# print(wordsD1)

hashtagV1 = []
hashtagV2 = []

count = 0
# print('\n#########   Printing ranking of hashtags   #########\n')
for key, value in sorted(iter(hashtag_dict.items()), reverse=True, key=lambda k_v4: (k_v4[1],k_v4[0])):
    if count < hashtag_limit:
        # print('%s: %s' % (key, value))
        hashtagV1.append(key)
        hashtagV2.append(value)
        
    count +=1

hashtagD1 = dict(zip(hashtagV1, hashtagV2))

# print(hashtagD1)

# plt.rcdefaults()
# fig, ax = plt.subplots()

# y_pos = np.arange(len(hashtagV1))
# error = np.random.rand(len(hashtagV1))

# ax.barh(y_pos, hashtagV2, xerr=error, align='center')
# ax.set_yticks(y_pos)
# ax.set_yticklabels(hashtagV1)
# ax.invert_yaxis()  # labels read top-to-bottom
# ax.set_xlabel('Frequência')
# ax.set_title('Hashtags mais utilizadas')

# plt.show()


usersV1 = []
usersV2 = []

count = 0
# print('\n#########   Printing ranking of most mentioned users   #########\n')
for key, value in sorted(iter(user_dict.items()), reverse=True, key=lambda k_v5: (k_v5[1],k_v5[0])):
    if count < user_limit:
        # print('%s: %s' % (key, value))
        usersV1.append(key)
        usersV2.append(value)
    count += 1


usersD1 = dict(zip(usersV1, usersV2))
# print(usersD1)

most_retweeted_list = []
list_is_full = False

for tweet in items:
    if 'rt @' in tweet['text']:
        pass
    elif tweet['retweet_count'] > 10:
        most_retweeted_list.append(tweet)

count = 0


tweets =[]
rts = []
ids = []

# print('\n#########   Printing most retweeted tweets   #########\n')
for tweet in sorted(most_retweeted_list, key = lambda x: x['retweet_count'], reverse=True):
    current_date = datetime.strptime(tweet['created_at'],'%Y-%m-%d %H:%M:%S+00:00')
    # if (current_date.year == 2020) and (current_date.month == 11) and (current_date.day < 17):
    tweets.append(tweet['screen_name'] +" - "+ tweet['text'])
    rts.append(tweet['retweet_count'])
    ids.append(tweet['id'])
    if count == 20:
        break
    count+=1

# rtKeys1 = ['tweet','rts','ids']
# rtKeys2 =  rtKeys1 *len(ids)
# print(rtKeys2)

retweetsD1 = [{'tweet': tweet, 'rts': rt, 'ids': idd} for tweet,rt,idd in zip(tweets,rts,ids)]



day_list = []
count = 0
for tweet in items:
    current_date = datetime.strptime(tweet['created_at'],'%Y-%m-%d %H:%M:%S+00:00')
    # if (current_date.year == 2020) and (current_date.month == 11) and (current_date.day < 17):
    date_time_str = str(current_date.day) +"/"+ str(current_date.month) +"/"+ str(current_date.year)
    # print(date_time_str)
    # day_list.append(str(current_date.day) +"/"+ str(current_date.month))
    date_time_obj = datetime.strptime(date_time_str, '%d/%m/%Y')
    # print(date_time_obj)
    day_list.append(date_time_obj)

day_dict = {}


for day in day_list:
    if day in day_dict:
        day_dict[day] += 1
    else:
        day_dict[day] = 1


DatasV1 = []
DatasV2 = []

# #going to break on cases that go from 23 to midnight or backwards
# print('\n#########   Printing hourly traffic   #########\n')
for key, value in sorted(iter(day_dict.items()), reverse=False, key=lambda k_v: (k_v[0],k_v[1])):
    chave = str(key)
    chave = re.sub(" 00:00:00","", chave)
    DatasV1.append(chave)
    DatasV2.append(value)
    # print('%s: %s' % (key, value))
DatasD1 = dict(zip(DatasV1, DatasV2))

# json.dump({report:[{'words':wordsD1},{'hashtags':hashtagD1},{'users':usersD1},{'retweets':retweetsD1},{'dates':DatasD1}]},pre_jsonReport, indent=4)

    

#### 2.31 - Gera a visualização

In [ ]:
axd = plt.figure(figsize=(6,10)).subplot_mosaic(
    """
    A
    B
    """,
    gridspec_kw={
        # 'width_ratios': [5, 5, 7],
        # 'height_ratios': [5, 5, 7],
        # 'right': 1.2,
        # 'left': 1,
        # 'hspace': 0.3
        
    })





# usersV1.reverse()
# usersV2.reverse()

# hashtagV1.reverse()
# hashtagV2.reverse()


axd['A'].barh(usersV1, usersV2, color='green')

for i, v in enumerate(usersV2):
    axd['A'].text(v-30, i-.07, str(v), color='white',size=12)



axd['B'].barh(hashtagV1, hashtagV2, color='purple')

for i, v in enumerate(hashtagV2):
    axd['B'].text(v-28, i-0.1, str(v), color='white',size=12)


axd['A'].set_yticklabels(usersV1,fontsize=13)
axd['B'].set_yticklabels(hashtagV1,fontsize=13)

   
# --------------------------------------------------------------------------


axd['A'].set_title('Usuários',fontsize=16)
axd['B'].set_title('Hashtags',fontsize=16)






# axd['A'].add_gridspec={'right': 100,'wspace': 30}
# plt.tight_layout()

# axd['C'].imshow(wordcloud, interpolation = 'bilinear')


plt.savefig("novos-graficos/Figura x - Usuários, hashtags e palavras mais frequentes no Twitter - Rio Grande do Sul.jpg",bbox_inches = 'tight')



### 2.32 - Palavras mais utilizadas nas proposições e tweets

In [ ]:
def tagclouds(perfil,ws,hs,arq):
    unik = list(usuarios_proposicoes[perfil].unique())
    tags = ["PLS_KEYWORDS", "TWEET_KEYWORDS", "INTERSECAO"]

    nomes = ['Projetos de Lei', 'Tweets', 'Interseção']

    axs1 = []

    fig = plt.figure(figsize=(20,20))
    # gs0 = matplotlib.gridspec.GridSpec(len(unik),len(tags), figure=fig)
    gs0 = matplotlib.gridspec.GridSpec(len(unik),len(tags), figure=fig,wspace=ws,hspace=hs)


    for i1,t in enumerate(unik):
        for i2,k in enumerate(tags):
            axs1.append(fig.add_subplot(gs0[i1,i2]))


    cores = ['#D38B07', '#3399FF','#ABAB5B']
    mxCores = [cores for k in unik]

    matrix = [tags for k in unik]
    indMatrix1 = [[i for i in range(len(tags))] for value in matrix]
    contagem = [i for i in range(len(tags)*len(unik))]
    indMatrix2 = [contagem[x:x+3] for x in range(0, len(contagem), 3)]



    f = open("stopwords.txt", 'r', encoding='utf-8')
    stopwords = [name.rstrip().lower() for name in f]



    # for idx1, t in enumerate(tags):
    for idx1, k in enumerate(unik):

        for idx2, t in enumerate(tags):
        # for idx2, k in enumerate(unik):
            wordcloud = WordCloud(
                # width = 50,
                # height = 50,
                prefer_horizontal=1,
                max_font_size=40,
                min_font_size=10,
                max_words=10,
                background_color = 'white',
                color_func=lambda *args, **kwargs: mxCores[idx1][idx2],
                # color_func=lambda *args, **kwargs: cores[idx2],
                stopwords=stopwords).generate(str(usuarios_proposicoes[usuarios_proposicoes[perfil]==k][t]))


            logro = []
            axs1[indMatrix2[idx1][idx2]].set_yticklabels(logro)
            axs1[indMatrix2[idx1][0]].set_ylabel(unik[idx1],fontsize=17)
            axs1[idx2].set_title(nomes[idx2],fontsize=17)

            axs1[indMatrix2[idx1][idx2]].tick_params(
            axis='both',          
            which='both',     
            bottom=False,      
            top=False,         
            # left=False,        
            # labelleft=False, 
            labelbottom=False) 


            axs1[indMatrix2[idx1][idx2]].imshow(wordcloud, interpolation = 'bilinear')
            # axs[idx].plot([-.1, 1.2], [0, 0], color='black', lw=1, transform=axs[idx].transAxes, clip_on=False)

            # axd['A'].add_gridspec={'right': 100,'wspace': 30}
            # plt.tight_layout()

        plt.savefig(arq,bbox_inches = 'tight')


In [ ]:
usuarios_proposicoes['GERAL'] = 'GERAL'

perfil = 'GERAL'
arq = "novos-graficos/Figura x - Nuvens de palavras sobre proposições e tweets - Geral.jpg"
tagclouds(perfil,0.1,-0.73,arq)

In [ ]:
def tagclouds2(perfil,ws,hs,arq):
    unik = list(filtro[perfil].unique())
    tags = ["PLS_KEYWORDS", "TWEET_KEYWORDS", "INTERSECAO"]

    nomes = ['Projetos de Lei', 'Tweets', 'Interseção']

    axs1 = []

    fig = plt.figure(figsize=(20,20))
    # gs0 = matplotlib.gridspec.GridSpec(len(unik),len(tags), figure=fig)
    gs0 = matplotlib.gridspec.GridSpec(len(unik),len(tags), figure=fig,wspace=ws,hspace=hs)


    for i1,t in enumerate(unik):
        for i2,k in enumerate(tags):
            axs1.append(fig.add_subplot(gs0[i1,i2]))


    cores = ['#D38B07', '#3399FF','#ABAB5B']
    mxCores = [cores for k in unik]

    matrix = [tags for k in unik]
    indMatrix1 = [[i for i in range(len(tags))] for value in matrix]
    contagem = [i for i in range(len(tags)*len(unik))]
    indMatrix2 = [contagem[x:x+3] for x in range(0, len(contagem), 3)]



    f = open("stopwords.txt", 'r', encoding='utf-8')
    stopwords = [name.rstrip().lower() for name in f]



    # for idx1, t in enumerate(tags):
    for idx1, k in enumerate(unik):

        for idx2, t in enumerate(tags):
        # for idx2, k in enumerate(unik):
            wordcloud = WordCloud(
                # width = 50,
                # height = 50,
                prefer_horizontal=1,
                max_font_size=40,
                min_font_size=10,
                max_words=10,
                background_color = 'white',
                color_func=lambda *args, **kwargs: mxCores[idx1][idx2],
                # color_func=lambda *args, **kwargs: cores[idx2],
                stopwords=stopwords).generate(str(filtro[filtro[perfil]==k][t]))


            logro = []
            axs1[indMatrix2[idx1][idx2]].set_yticklabels(logro)
            axs1[indMatrix2[idx1][0]].set_ylabel(unik[idx1],fontsize=17)
            axs1[idx2].set_title(nomes[idx2],fontsize=17)

            axs1[indMatrix2[idx1][idx2]].tick_params(
            axis='both',          
            which='both',     
            bottom=False,      
            top=False,         
            # left=False,        
            # labelleft=False, 
            labelbottom=False) 


            axs1[indMatrix2[idx1][idx2]].imshow(wordcloud, interpolation = 'bilinear')
            # axs[idx].plot([-.1, 1.2], [0, 0], color='black', lw=1, transform=axs[idx].transAxes, clip_on=False)

            # axd['A'].add_gridspec={'right': 100,'wspace': 30}
            # plt.tight_layout()

        plt.savefig(arq,bbox_inches = 'tight')

In [ ]:
arq = "novos-graficos/Figura x - Nuvens de palavras sobre proposições e tweets - Rio Grande do Sul.jpg"
filtro =usuarios_proposicoes[usuarios_proposicoes['ESTADO_ELEICAO'].isin(['RS'])]
perfil = 'ESTADO_ELEICAO' 

tagclouds2(perfil,0.1,-0.5,arq)

# 3 - Formação de redes

### 3.1 - Carrega bases de dados

In [ ]:
listaDeputados = pd.read_csv("db-listaDeputados.csv")

usuarios_proposicoes = pd.read_csv("usuarios_proposicoes-2.csv")
tweets2020 = pd.read_csv("tweets2020.csv")
# usuarios_proposicoes.head(2)
usuarios_proposicoes_rs = usuarios_proposicoes[(usuarios_proposicoes['ESTADO_ELEICAO']=='RS')]

votacoes = pd.read_csv("db-votacoes.csv")


usuarios_proposicoes_rs.to_csv("db-usuarios-rs.csv", index=False)

# len(tweets2020)
proposicoes = pd.read_csv("db-proposicoes.csv")

### 3.2 - Junta os tweets com as informações demográficos dos seus autores

In [ ]:
tweets2020_info = tweets2020.merge(listaDeputados, left_on='screen_name', right_on='PERFIL_TWITTER')
tweets2020_info_rs = tweets2020_info[(tweets2020_info['ESTADO_ELEICAO']=='RS')]
# len(tweets2020_info_rs)

## 3.3 - Proposições com maior percentual de aprovação e rejeição

In [ ]:
votacoes["TITULO_PROPOSICAO"] = votacoes["TITULO_PROPOSICAO"].str.replace("\((.*?)\)","")

votacoes["TITULO_PROPOSICAO"] = votacoes["TITULO_PROPOSICAO"].str.strip()

In [ ]:
votacoes_c = votacoes[["TITULO_PROPOSICAO","VOTOS_SIM"]]
votacoes_c = votacoes_c.drop_duplicates(subset='TITULO_PROPOSICAO')

votacoes_c10 = votacoes_c.nlargest(5,'VOTOS_SIM').sort_values(by='VOTOS_SIM', ascending=True)



votacoes_c10['percentual'] = votacoes_c10.apply(lambda row: round(row.VOTOS_SIM/513*100,2), axis=1)

votacoes_c10 = votacoes_c10.set_index('TITULO_PROPOSICAO')

votacoes_c10['rotulos'] = votacoes_c10.apply(lambda row: str(int(row.VOTOS_SIM)) +" ("+ str(row.percentual)+"%) " , axis=1)

votacoes_s10 = votacoes_c10['percentual']
# type(votacoes_c10)


In [ ]:
votacoes_b = votacoes[["TITULO_PROPOSICAO","VOTOS_NAO"]]
# votacoes_b = votacoes_b.drop_duplicates(subset='TITULO_PROPOSICAO')

votacoes_b10 = votacoes_b.nlargest(5,'VOTOS_NAO').sort_values(by='VOTOS_NAO', ascending=True)



votacoes_b10['percentual'] = votacoes_b10.apply(lambda row: round(row.VOTOS_NAO/513*100,2), axis=1)

votacoes_b10 = votacoes_b10.set_index('TITULO_PROPOSICAO')

votacoes_b10['rotulos'] = votacoes_b10.apply(lambda row: str(int(row.VOTOS_NAO)) +" ("+ str(row.percentual)+"%) " , axis=1)

votacoes_n10 = votacoes_b10['percentual']
# type(votacoes_b10)


In [ ]:
axd = plt.figure(figsize=(8,10)).subplot_mosaic(
    """
    A
    B
    """
)

top10 = votacoes_s10.plot(kind='barh', 
                        color='green', 
                        width=0.8,ax=axd['A'])


indices1 = list(votacoes_c10['rotulos'])

for i, v in enumerate(list(votacoes_s10)):
    top10.text(v-18, i, indices1[i], color='white',weight='heavy')
    
    
    
    
    

bottom10 = votacoes_n10.plot(kind='barh', 
                        color='#BC0D0D', 
                        width=0.8,ax=axd['B'])

indices2 = list(votacoes_b10['rotulos'])

for i, v in enumerate(list(votacoes_n10)):
    bottom10.text(v-18, i, indices2[i], color='white',weight='heavy')
    
    
top10.set_ylabel('')    
top10.set_xlabel('')

bottom10.set_ylabel('')    
bottom10.set_xlabel('')

bottom10.grid(color='b', linestyle='-', linewidth=.1)   
top10.grid(color='b', linestyle='-', linewidth=.1)   

plt.tight_layout()
# axd

# plt.show()

# plt.savefig("novos-graficos/Figura x - Índices de consistência temática por estado.jpg",bbox_inches = 'tight')
# plt.clf()
# plt.show()

plt.savefig("novos-graficos/Figura x - Proposições com maiores percentuais de aprovação e rejeição.jpg",bbox_inches = 'tight')


## 3.4 - Rede de coautoria de proposições parlamentares

### 3.5 - Criação do Grafo

In [ ]:
%%capture

arq_grafo = "db-proposicoes.csv"
arq_semantico = "grafo-coautoria.csv"
df_col3 = pd.read_csv(arq_grafo)
# writer.writerow(['TITULO_FRENTE','DEPUTADOS_INTEGRANTES'])
# writer = csv.writer(fileOut)
# reader = csv.reader(fileIn, delimiter=',')   
with open(arq_grafo, encoding="utf-8") as f:
    vTweets = [row["NOME_AUTOR"] for row in DictReader(f)]

col1 = []
col2 = []
vcol3 = []

for tweet in vTweets:
    # print(tweet)
    vPalavras = tweet.split(',')
    nPalavras = len(vPalavras)

    if nPalavras > 2:
        nLinhas = 0
        vNumeros = []
        for i in range(nPalavras):
            b = nPalavras-(i+1)
            if b>0:
                vNumeros.append(b)
            nLinhas = nLinhas + b
        vcol3.append(nLinhas)

        nNumeros= len(vNumeros)
        invNumeros = vNumeros[::-1]
        
        c1 = []
        for i in range(nNumeros):
            for j in range(vNumeros[i]):
                c1.append(vPalavras[i])  
        col1.extend(c1)

 
        ordemC2 = []
        for i in range(nNumeros):
            for j in invNumeros:
                ordemC2.append(j)
            invNumeros.pop(0)

        Mposicao = []
        posicaoC2 = []
        for i in range(int(nLinhas/nPalavras)):
            for j in range(nPalavras):
                posicaoC2.append(vPalavras[j])
                Mposicao.append(j)

        c2 =[]
        c2[:] = [posicaoC2[i] for i in ordemC2]
        col2.extend(c2)



scol3 = df_col3[['TITULO_PROPOSICAO','TEMA', 'PALAVRAS_CHAVE','TIPO_AUTOR']].values.tolist()

col3 = [c for c, i in zip(scol3, vcol3) for _ in range(i)]

# print(len(col3))
# print(len(col3))
matriz = np.c_[col1,col2,col3]
df = pd.DataFrame(columns=["SOURCE","TARGET",'TITULO_PROPOSICAO','TEMA', 'PALAVRAS_CHAVE','TIPO_AUTOR'], data=matriz)
df['SOURCE'].replace('', np.nan, inplace=True)
df.dropna(subset=['SOURCE'], inplace=True)
df['TARGET'].replace('', np.nan, inplace=True)
df.dropna(subset=['TARGET'], inplace=True)

df.to_csv(arq_semantico, index=False)

### 3.6 - Filtro do Grafo

In [ ]:
coautoria_r = pd.read_csv("grafo-coautoria.csv")
# len(proposicoes)

# len(coautoria_r)
coautoria_f = coautoria_r[(coautoria_r['TIPO_AUTOR']=='Deputado')]
# len(coautoria_f)

coautoria_info = coautoria_f.merge(usuarios_proposicoes, left_on='SOURCE', right_on='NOME_AUTOR',how='inner')

# coautoria_info.shape[0]
# list(coautoria_info.columns)

keepcol = ['SOURCE','TARGET','TITULO_PROPOSICAO_x','TEMA_x','PALAVRAS_CHAVE','TIPO_AUTOR','NOME_AUTOR','N_PLS','GENERO','COR_RACA','ESTADO_CIVIL','ESCOLARIDADE','OCUPACAO_ANTERIOR','FAIXA_ETARIA','PARTIDO_ATUAL','IDEOLOGIA_PARTIDO_ATUAL','MILIONARIO','ESTADO_ELEICAO','REGIAO_ELEICAO','ST_REELEICAO','AGRUPAMENTO_IDADE','AGRUPAMENTO_BENS','AGRUPAMENTO_VOTOS','AGRUPAMENTO_SEGUIDORES','AGRUPAMENTO_BALANCO_SEGUIDORES','AGRUPAMENTO_NUMERO_TWEETS','AGRUPAMENTO_BALANCO_TWEETS','TEM_TWITTER','TOP_BTWEETS','TOP_BSEG','TOP_TWEETS','TOP_SEG','TOP_VOTACAO','TOP_BENS','SCREEN_NAME','TWEET_COUNTS','MEDIA_RETWEETS','SOMA_RETWEETS','MEDIA_FAV','SOMA_FAV','CONSISTENCIA_TEMATICA','AGRUPAMENTO_cTEMATICA']
 
coautoria_info = coautoria_info[keepcol]

coautoria_info = coautoria_info.rename(columns={'SOURCE':'source','TARGET':'target'})

coautoria_info.to_csv("grafo-coautoria.csv", index=False)
arq_semantico = "grafo-coautoria.csv"
# len(tweets2020_info_rs)


coautoria_info_rs= coautoria_info[(coautoria_info['ESTADO_ELEICAO']=='RS')]
# len(coautoria_info_rs)

coautoria_info_rs.to_csv("grafo-coautoria-rs.csv", index=False)
arq_semantico = "grafo-coautoria-rs.csv"

coautoria_rs = pd.read_csv("grafo-coautoria-rs.csv")
lista_rs = list(coautoria_rs['source'].unique())


coautoria_f_rs = coautoria_rs.query('target in @lista_rs')
coautoria_f_rs.to_csv("grafo-coautoria-f-rs.csv", index=False)

In [ ]:
len(lista_rs)

### 3.7 - Métricas

In [ ]:
grafo = 'grafo-coautoria-f-rs.csv'

df = pd.read_csv(grafo)
Graphtype = nx.Graph()
G = nx.from_pandas_edgelist(df, create_using=Graphtype)

Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])


G.number_of_nodes()
G.number_of_edges()


In [ ]:
nx.density(G)

In [ ]:
c = list(greedy_modularity_communities(G))
# len(c)
# c

In [ ]:
degree_centrality = dict(nx.degree(G))
# dict(degree_centrality)
df = pd.DataFrame.from_dict(degree_centrality, orient='index')

df.index.name = 'deputado'
df= df.reset_index()
df.rename(columns = {0:'grau'},inplace = True)


df_info = pd.merge(df, usuarios_proposicoes_rs[['NOME_AUTOR','PARTIDO_ATUAL','IDEOLOGIA_PARTIDO_ATUAL','ST_REELEICAO','ESTADO_ELEICAO','REGIAO_ELEICAO','N_PLS','TWEET_COUNTS','MEDIA_RETWEETS','AGRUPAMENTO_BENS', 'AGRUPAMENTO_VOTOS', 'AGRUPAMENTO_SEGUIDORES','AGRUPAMENTO_NUMERO_TWEETS','TOP_TWEETS', 'TOP_SEG', 'TOP_VOTACAO', 'TOP_BENS']], left_on='deputado', right_on='NOME_AUTOR',how='inner')

df_info = df_info.drop(['NOME_AUTOR'],axis=1)

df_info = df_info.sort_values(by='grau', ascending=False)

df_info= df_info.reset_index()

df_info = df_info.drop(['index'],axis=1)
# df_info[:10]
df_info

# df_info['N_PLS'].mean()



In [ ]:
# df = pd.DataFrame.from_dict(betweenness_centrality, orient='index')

betweenness_centrality = dict(nx.betweenness_centrality(G,normalized=False))
# degree_centrality = dict(nx.degree(G))
# dict(degree_centrality)
df = pd.DataFrame.from_dict(betweenness_centrality, orient='index')

df.index.name = 'deputado'
df= df.reset_index()
df.rename(columns = {0:'intermediacao'},inplace = True)


df_info = pd.merge(df, usuarios_proposicoes_rs[['NOME_AUTOR','PARTIDO_ATUAL','IDEOLOGIA_PARTIDO_ATUAL','ST_REELEICAO','ESTADO_ELEICAO','REGIAO_ELEICAO','N_PLS','TWEET_COUNTS','MEDIA_RETWEETS','AGRUPAMENTO_BENS', 'AGRUPAMENTO_VOTOS', 'AGRUPAMENTO_SEGUIDORES','AGRUPAMENTO_NUMERO_TWEETS','TOP_TWEETS', 'TOP_SEG', 'TOP_VOTACAO', 'TOP_BENS']], left_on='deputado', right_on='NOME_AUTOR',how='inner')

df_info = df_info.drop(['NOME_AUTOR'],axis=1)

df_info = df_info.sort_values(by='intermediacao', ascending=False)

df_info= df_info.reset_index()

df_info = df_info.drop(['index'],axis=1)
# df_info[:10]
df_info


## 3.8 - Menção a proposições no Twitter

### 3.9 - Filtra tweets por menções de proposições

In [ ]:
pat = 'projeto de lei|requerimento de informação|projeto de decreto legislativo|projeto de lei complementar|medida provisória|projeto de resolução de alteração do regimento|projeto de lei de conversão|emenda|substitutivo do senado|proposta de emenda à constituição|solicitação de informação ao tcu|mensagem de indicação de líder|consulta|proposta de fiscalização e controle|requerimento de instituição de cpi| pl | ric  | inc | pdl | plp | mpv | prc | plv | ems | pec | sit | msc | con | pfc | rcp | projeto | proposicao'



filtro_geral =tweets2020_info[tweets2020_info['text'].str.contains(pat)]

filtro_rs =tweets2020_info_rs[tweets2020_info_rs['text'].str.contains(pat)]



# tweets2020

In [ ]:
# filtro_rs.to_csv("contsulta_rs.csv")
# filtro_geral.to_csv("contsulta_geral.csv")

### 4.0 - Calcula a porcentagem para o âmbito geral

In [ ]:
len(tweets2020_info)
len(filtro_geral)
uno = len(tweets2020_info)
dos = len(filtro_geral)
round(dos/uno*100,2)

In [ ]:
len(filtro_geral['NM_URNA'].unique())

### 4.5 - Calcula a porcentagem para deputados gaúchos

In [ ]:
len(tweets2020_info_rs)
len(filtro_rs)
uno = len(tweets2020_info_rs)
dos = len(filtro_rs)
round(dos/uno*100,2)

In [ ]:
len(filtro_rs['NM_URNA'].unique())

In [ ]:
rgx = 'projeto de lei\s\d*|requerimento de informação\s\d*|indicação\s\d*|projeto de decreto legislativo\s\d*|projeto de lei complementar\s\d*|medida provisória\s\d*|projeto de resolução de alteração do regimento\s\d*|projeto de lei de conversão\s\d*|emenda\s\d*|substitutivo do senado\s\d*|proposta de emenda à constituição\s\d*|solicitação de informação ao tcu\s\d*|mensagem de indicação de líder\s\d*|consulta\s\d*|proposta de fiscalização e controle\s\d*|requerimento de instituição de cpi\s\d*|pl\s\d*|ric \s\d*|inc\s\d*|pdl\s\d*|plp\s\d*|mpv\s\d*|prc\s\d*|plv\s\d*|ems\s\d*|pec\s\d*|sit\s\d*|msc\s\d*|con\s\d*|pfc\s\d*|rcp\s\d*|proposicao\s\d*|projeto\s\d*|lei\s\d*\.\d+'

lista_rs  = list(tweets2020_info_rs['text'].apply(unidecode).str.lower())

lista_geral = list(tweets2020_info['text'].apply(unidecode).str.lower())

In [ ]:
# newlist = list(filter(r.match, lista_rs)) 
# r = re.compile(rgx2)

str_list_rs = " ".join(lista_rs)
str_list_geral = " ".join(lista_geral)

etapa_rs = re.findall(rgx,str_list_rs)
etapa_geral = re.findall(rgx,str_list_geral)

resultados_rs = [i for i in etapa_rs if any(map(str.isdigit, i))]
resultados_geral = [i for i in etapa_geral if any(map(str.isdigit, i))]

In [ ]:
len(resultados_geral)
# resultados_geral
len(resultados_rs)


### 4.6 - Gera Gráfico com menções a proposições - Geral

In [ ]:
resultados_rs = [i.strip() for i in resultados_rs]
resultados_geral = [i.strip() for i in resultados_geral]

contagem_geral = Counter(resultados_geral)
contagem_geral = dict(contagem_geral)
df = pd.DataFrame.from_dict(contagem_geral, orient='index')



df = df.nlargest(6,[0])
# df.nlargest(12,df[0]).sort_values(by=df[0], ascending=False)

contagem_geral = df[0]
# contagem_geral.drop([0])
# contagem_geral.drop(contagem_geral.index[0])
# type(contagem_geral)
contagem_geral = contagem_geral.iloc[1:]
contagem_geral = contagem_geral.sort_values(ascending=True)
# contagem_geral


In [ ]:
                        
top = contagem_geral.plot(kind='barh', 
                        colormap='tab10', 
                        figsize=(11, 8),width=0.8)

for i, v in enumerate(list(contagem_geral)):
    top.text(v+.1, i-0.1, v ,fontsize=15, color='black')

plt.tick_params(axis='y', which='major', labelsize=14)
    
# top.margins(x=1)
# top.spines['right'].set_visible(False)
# top.spines['top'].set_visible(False)
# top.spines['bottom'].set_visible(False)
    
# top.get_legend().remove()            

# plt.rcParams.update(IPython_default);
# plt.style.use("fivethirtyeight")
plt.grid(color='b', linestyle='-', linewidth=.1)    
plt.savefig("novos-graficos/Figura x - Menção a proposições no Twitter- Geral.jpg",bbox_inches = 'tight')
# plt.show()

### 4.7 - Gera Gráfico com menções a proposições - Rio Grande do Sul

In [ ]:
contagem_rs = Counter(resultados_rs)
contagem_rs = dict(contagem_rs)
df = pd.DataFrame.from_dict(contagem_rs, orient='index')



df = df.nlargest(7,[0])
# df.nlargest(12,df[0]).sort_values(by=df[0], ascending=False)

contagem_rs = df[0]
# contagem_rs.drop([0])
# contagem_rs.drop(contagem_rs.index[0])
# type(contagem_rs)
contagem_rs = contagem_rs.iloc[1:]
contagem_rs = contagem_rs.sort_values(ascending=True)
contagem_rs = contagem_rs.drop(['pl 22'])
contagem_rs
# contagem_rs



In [ ]:
                        
top = contagem_rs.plot(kind='barh', 
                        colormap='tab10', 
                        figsize=(11, 8),width=0.8)

for i, v in enumerate(list(contagem_rs)):
    top.text(v+0.08, i-0.1, v ,fontsize=15, color='black')

plt.tick_params(axis='y', which='major', labelsize=14)
    
# top.margins(x=1)
# top.spines['right'].set_visible(False)
# top.spines['top'].set_visible(False)
# top.spines['bottom'].set_visible(False)
    
# top.get_legend().remove()            

# plt.rcParams.update(IPython_default);
# plt.style.use("fivethirtyeight")
plt.grid(color='b', linestyle='-', linewidth=.1)    
plt.savefig("novos-graficos/Figura x - Menção a proposições no Twitter- Rio Grande do Sul.jpg",bbox_inches = 'tight')
# plt.show()

In [ ]:
### Grafo de conexão do twitter filtrando por PLS - geral

In [ ]:
from collections import Counter

In [ ]:
filtro_geral.to_csv("filtro-geral.csv", encoding="utf-8", index=False)

filtro_rs.to_csv("filtro-rs.csv",encoding="utf-8", index=False)

In [ ]:
list(filtro_geral.columns)

## 4.8 - Rede de Tweets mencionando proposições 

### 4.9 - Grafo

In [ ]:
%%capture
arq = "filtro-geral"

arq_json = arq + ".json"
arq_csv = arq + ".csv"
arq_grafo = arq + "-grafo.csv"
arq_rt = arq + "-grafo-rt.csv"
arq_mt = arq + "-grafo-mt.csv"

outrow = []
with open(arq_csv, "r",encoding="utf-8") as fileIn:  # input file location
    with open(arq_grafo, "w") as fileOut:  # output file location
        writer = csv.writer(fileOut)
        reader = csv.reader(fileIn, delimiter=',')
        writer.writerow(['source','target', "text"])
        for row in reader:
            screen_name = row[0]    
            text = row[1]      
            target = ''
            for cell in row:
                target = re.findall(r"@([^\s]+)", cell)
                target = re.sub(r"\[|\'|\:|\]", '', str(target))
                outrow 
            writer.writerow([screen_name, target, text])

df = pd.read_csv(arq_grafo)
df['target'].replace('', np.nan, inplace=True)
df.dropna(subset=['target'], inplace=True)
df.to_csv(arq_grafo, index=False)

df = pd.read_csv(arq_grafo)
rt = df['text'].str.startswith('RT @')
df[rt].to_csv(arq_rt, index=False)
df2 = pd.read_csv(arq_rt)
df2['target'].replace(r",[\s\S]*$", "", regex=True, inplace= True)
df2.to_csv(arq_rt, index=False)


def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = df['target'].str.split(',').map(len)

res = pd.DataFrame({'source': np.repeat(df['source'], lens),
                    'target': chainer(df['target']),
                    'text': np.repeat(df['text'], lens)
                    })

res['target'].replace('', np.nan, inplace=True)
res.dropna(subset=['target'], inplace=True)

mt = ~res['text'].str.startswith('RT @')
res[mt].to_csv(arq_mt, index=False)
# os.remove(arq_grafo)


In [ ]:
# tweets2020[(tweets2020['screen_name']=='lpbragancabr')]
# len(filtro_geral['NM_URNA'].unique())

#### 4.10 - Métricas

In [ ]:
grafo = 'filtro-geral-grafo.csv'

df = pd.read_csv(grafo)
Graphtype = nx.DiGraph()
G = nx.from_pandas_edgelist(df, create_using=Graphtype)

# Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
# G = G.subgraph(Gcc[0])
G.remove_nodes_from(list(nx.isolates(G)))

G.number_of_nodes()
G.number_of_edges()


In [ ]:
nx.density(G)

In [ ]:
degree_centrality = dict(G.in_degree())
# dict(degree_centrality)
df = pd.DataFrame.from_dict(degree_centrality, orient='index')

df.index.name = 'deputado'

df= df.reset_index()
df.rename(columns = {0:'grau'},inplace = True)



df_info = pd.merge(df, usuarios_proposicoes[['SCREEN_NAME','NOME_AUTOR','PARTIDO_ATUAL','IDEOLOGIA_PARTIDO_ATUAL','ST_REELEICAO','ESTADO_ELEICAO','REGIAO_ELEICAO','N_PLS','TWEET_COUNTS','MEDIA_RETWEETS','AGRUPAMENTO_BENS', 'AGRUPAMENTO_VOTOS', 'AGRUPAMENTO_SEGUIDORES','AGRUPAMENTO_NUMERO_TWEETS','TOP_TWEETS', 'TOP_SEG', 'TOP_VOTACAO', 'TOP_BENS']], left_on='deputado', right_on='SCREEN_NAME',how='inner')

# df_info

df_info = df_info.drop(['SCREEN_NAME'],axis=1)

df_info = df_info.sort_values(by='grau', ascending=False)
# df_info.shape[0]
df_info

In [ ]:
df_info_rs = df_info[(df_info['ESTADO_ELEICAO']=='RS')]
df_info_rs

In [ ]:
# df = pd.DataFrame.from_dict(betweenness_centrality, orient='index')

betweenness_centrality = dict(nx.betweenness_centrality(G,normalized=False))
# degree_centrality = dict(nx.degree(G))
# dict(degree_centrality)
df = pd.DataFrame.from_dict(betweenness_centrality, orient='index')

df.index.name = 'deputado'
df= df.reset_index()
df.rename(columns = {0:'intermediacao'},inplace = True)

df_info = pd.merge(df, usuarios_proposicoes[['SCREEN_NAME','NOME_AUTOR','PARTIDO_ATUAL','IDEOLOGIA_PARTIDO_ATUAL','ST_REELEICAO','ESTADO_ELEICAO','REGIAO_ELEICAO','N_PLS','TWEET_COUNTS','MEDIA_RETWEETS','AGRUPAMENTO_BENS', 'AGRUPAMENTO_VOTOS', 'AGRUPAMENTO_SEGUIDORES','AGRUPAMENTO_NUMERO_TWEETS','TOP_TWEETS', 'TOP_SEG', 'TOP_VOTACAO', 'TOP_BENS']], left_on='deputado', right_on='SCREEN_NAME',how='inner')

# df_info

df_info = df_info.drop(['SCREEN_NAME'],axis=1)

df_info = df_info.sort_values(by='intermediacao', ascending=False)
# df_info.shape[0]
df_info


In [ ]:
df_info_rs = df_info[(df_info['ESTADO_ELEICAO']=='RS')]
df_info_rs

### 4.11 - Publicações mais compartilhadas

#### 4.12 - Geral

In [ ]:
# filtro_geral.head(2)
topTweets = filtro_geral.nlargest(3,'retweet_count')
# tweetsTop10
topIds = topTweets['id'].astype("str").values.tolist()

string = "https://twitter.com/OReillyMedia/status/"

topIds = [string + s for s in topIds]
# topIds

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text

# Tweet("https://twitter.com/OReillyMedia/status/1253710046405500928")
    
for x in topIds:
    Tweet(x)
# vTweets(topIds)
    

#### 4.13 - Rio Grande do Sul

In [ ]:
topTweets = filtro_rs.nlargest(3,'retweet_count')
# tweetsTop10
topIds = topTweets['id'].astype("str").values.tolist()

string = "https://twitter.com/OReillyMedia/status/"

topIds = [string + s for s in topIds]
# topIds

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text

# Tweet("https://twitter.com/OReillyMedia/status/1253710046405500928")
    
for x in topIds:
    Tweet(x)
# vTweets(topIds)